# Data Mining Challenge 2020-2021

## Team
* Francesco Fulco Gonzales 10614882
* Francesco Govigli 10556637
* Alberto Latino 10600138


In [18]:
import pandas as pd
import numpy as np
import copy
import random

import matplotlib.pyplot as plt
import seaborn as sns
from dython.nominal import associations
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

# Classification methods
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from dython.nominal import associations
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.model_selection import train_test_split
from scipy import stats

from IPython.core.interactiveshell import InteractiveShell  # print all outputs
InteractiveShell.ast_node_interactivity = "all"

In [19]:
SEED = 1234
random.seed(SEED)

In [20]:
df_test = pd.read_csv("data/test.csv")

In [21]:
df_test.info(verbose=True, null_counts=True) # there are no null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160200 entries, 0 to 160199
Data columns (total 135 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   SITE_ID                              160200 non-null  int64  
 1   DATE                                 160200 non-null  object 
 2   CELL_TYPE_Macro                      160200 non-null  int64  
 3   CELL_TYPE_Mobil                      160200 non-null  int64  
 4   CELL_TYPE_TRP                        160200 non-null  int64  
 5   CELL_TYPE_Tx site                    160200 non-null  int64  
 6   CELL_TYPE_micro                      160200 non-null  int64  
 7   N_TRANSPORTED_SITES                  160200 non-null  float64
 8   GEOGRAPHIC_CLUSTER_K_0               160200 non-null  int64  
 9   GEOGRAPHIC_CLUSTER_K_1               160200 non-null  int64  
 10  GEOGRAPHIC_CLUSTER_K_2               160200 non-null  int64  
 11  GEOGRAPHIC_C

### CELL_TYPE

In [22]:
cell_types = ["CELL_TYPE_Macro","CELL_TYPE_Mobil","CELL_TYPE_TRP","CELL_TYPE_Tx site","CELL_TYPE_micro"]

In [23]:
# get total number of sites and number of faulty sites for each cell type 
df_cell_total = df_test[cell_types].sum().to_frame(name='tot count')

In [24]:
# Since CELL_TYPE_TRP is not contained in any trasmission site, we drop this column since it doesn't add useful information
df_test.drop('CELL_TYPE_TRP', inplace=True, axis=1)
cell_types.remove('CELL_TYPE_TRP')

### Cleaning Null Values

#### We noticed that both in the dataset and in the testset there are samples with no cell type. It should be wrong because every site should have at least one cell to transmit signals. Therefore, we imputed these data from the dataset. We substituted this "missing" value" with the most frequent cell type (Macro).

In [25]:
#We want to verify how many types of cell are supported for each site. 
# The sum can be: 0 if no cell type is supported, 1 if it has one cell type, > 1 if it has more than one cell type
sub_df = df_test[cell_types]

# rows without celltype 
no_cell_df = sub_df.sum(axis=1) == 0
print("There are", no_cell_df[no_cell_df == True].shape[0], "sites without cell type")

# rows with more than one celltype 
many_cell_df = sub_df.sum(axis=1) > 1
print("There are",many_cell_df[many_cell_df == True].shape[0], "sites with more than one cell type")

There are 900 sites without cell type
There are 0 sites with more than one cell type


In [26]:
#We impute the cell type by inserting the most frequent one
indexes = df_test[df_test[cell_types].sum(axis=1) == 0].index
df_test.loc[df_test[cell_types].sum(axis=1) == 0, ["CELL_TYPE_Macro"]] = 1

In [27]:
# Let's aggregate the CELL_TYPE attributes, by creating a unique column 
# This columns contains a value from 0 to 3 that represent the belonging cell type

dummies = pd.get_dummies(df_test[cell_types])
df_test['CELL_TYPE'] = dummies.cumsum(axis=1).ne(1).sum(axis=1)

df_test["CELL_TYPE"] = LabelEncoder().fit_transform(df_test["CELL_TYPE"])

df_test.drop(cell_types, inplace=True, axis=1)

In [28]:
# Reorders columns
col = df_test['CELL_TYPE']
df_test.pop('CELL_TYPE')
df_test.insert(df_test.columns.get_loc('N_TRANSPORTED_SITES') + 1, col.name, col, allow_duplicates=True)

0         0
1         0
2         0
3         0
4         0
         ..
160195    0
160196    0
160197    0
160198    0
160199    0
Name: CELL_TYPE, Length: 160200, dtype: int64

### GEOGRAPHIC_CLUSTER

In [29]:
# Check if there are some sites that do not belong to any geogrhaphical cluster
# As a result all tuple has one unique geographic site because the num. of tuples for such that the sum of
#GEOGRAPHIC_CLUSTER_K is 1 are all the tuple 621300
geo_clusters = []
for i in range(10):
    geo_clusters.append("GEOGRAPHIC_CLUSTER_K_" + str(i))
    
sub_df = df_test[geo_clusters]
sub_df = sub_df.sum(axis=1) == 1

sub_df[sub_df == True].shape[0]

160200

In [30]:
# Let's aggregate the GEOGRAPHIC_CLUSTER attributes, by creating a unique column 
# This columns contains a value from 0 to 9 that represent the belonging cluster

df_geo = df_test[geo_clusters].groupby(
    df_test[geo_clusters].columns.str.split('_K_').str[0], axis=1).apply(
    lambda x: x.dot(x.columns.str.split('_K_').str[1])
)

In [31]:
excluded_attr = geo_clusters 
df_test = df_test.drop(excluded_attr,axis=1)

In [32]:
# Add aggregated column
idx = 7
df_test.insert(loc=idx, column='GEOGRAPHIC_CLUSTER', value=df_geo)

### *cat*_sum_alarms_prevXd

Number of alarms associated to the category *cat* observed in previous X days, where X = 3, 7, 14.


Alarms are classified in the following categories:

|Alarm category | Description  |
|---|---|
|  equipment | Alarms associated to faults on air conditioning equipments  |
| fire/smoke |  Alarms associated to the presence of fire/smoke on site |
| ge |  Alarms associated to faults on electric generator mounted on the site |
| power | Alarms associated to faults on power transmission unit  |
| temperature  | Alarms associated to problematic temperature conditions (high/low)  |


In [33]:
# get all sum_alarms attributes
alarm_types = ['equipment', 'fire/smoke', 'ge', 'power', 'temperature']
sum_alarms = []
for d in ['14', '7', '3']:
    for a in alarm_types:
        sum_alarms.append(a + '_sum_alarms_prev'+ d +'d')

In [34]:
# check if there are negative values
(df_test[sum_alarms] < 0).values.any()

False

In [35]:
# check that for each row, sum alarms of 14d >= 7d >= 3d
for a in alarm_types:
    attr = a + '_sum_alarms_prev'
    df_shape = df_test[(df_test[attr + '14d'] < df_test[attr + '7d']) 
                        &  (df_test[attr + '7d'] <  df_test[attr + '3d'])].shape[0]
    if(df_shape != 0):
        print("Prev days data of " + a + " are inconsistent")
    else:
        print("Prev days data of " + a + " are consistent")

Prev days data of equipment are consistent
Prev days data of fire/smoke are consistent
Prev days data of ge are consistent
Prev days data of power are consistent
Prev days data of temperature are consistent


 ### *cat*_mean/max/min_persistance_prevXd
 
 Mean, max or min alarm duration (in minutes) of *cat* alarms in the previous X days, where X = 3, 7.

In [36]:
# check that for each row min7d <= min3d and max7d >= max3d
for a in alarm_types:
    max_attr = a + '_max_persistance_prev'
    min_attr = a + '_min_persistance_prev'
    
    num_inconsistent = df_test[(df_test[max_attr + '7d'] < df_test[max_attr + '3d']) 
                        & (df_test[min_attr + '7d'] >  df_test[min_attr + '3d'])].shape[0]
    if(num_inconsistent != 0):
        print("Inconsistent prev days data for " + a)
    else:
        print("Prev days data of " + a + " are consistent")

Prev days data of equipment are consistent
Prev days data of fire/smoke are consistent
Prev days data of ge are consistent
Prev days data of power are consistent
Prev days data of temperature are consistent


In [37]:
# check that for each persistance in min it is not greater than the tot available minutes
MAX_7d = 7 * 24 * 60 # minutes in 7 days
MAX_3d = 3 * 24 * 60 # minutes in 3 days

outliers7d = {}
outliers3d = {}
indexToDrop = []

for attr in df_test.columns:
    if('persistance_prev7d' in attr):
        outliers7d[attr] = df_test[attr][df_test[attr] > MAX_7d].values
        indexToDrop = np.concatenate((indexToDrop,df_test[attr][df_test[attr] > MAX_7d].index), axis=0)
    elif('persistance_prev3d' in attr): 
        outliers3d[attr] = df_test[attr][df_test[attr] > MAX_3d].values
        indexToDrop = np.concatenate((indexToDrop,df_test[attr][df_test[attr] > MAX_3d].index), axis=0)

In [38]:
#Here we create a dataframe without outliers in order to compute the mean
#that we will use for imputation on outliers
df_no_outliers = df_test.drop(indexToDrop)

In [39]:
#For each outlier point, we set the respective mean
for attr in df_test.columns:
    if('persistance_prev7d' in attr):
        df_test.loc[df_test[attr] > MAX_7d, attr] = df_no_outliers[attr].mean()
    elif('persistance_prev3d' in attr): 
        df_test.loc[df_test[attr] > MAX_3d, attr] = df_no_outliers[attr].mean()

In [40]:
def outliers_Array(dic):
    values_arr = []
    for name in dic:
        values_arr = np.concatenate((values_arr, dic[name]), axis=0)
    return values_arr    

In [41]:
# We can drop the tuples containing the outlier values
df_test.drop(indexToDrop, inplace=True)

## Saving the Preprocessed Test Data


In [42]:
df_test.to_csv('data/processed_test.csv', index=False)

In [43]:
df_test

,SITE_ID,DATE,N_TRANSPORTED_SITES,CELL_TYPE,aircon_sum_wo_prev7d,aircon_sum_wo_prev14d,mean_temperature_prev7d,GEOGRAPHIC_CLUSTER,max_temperature_prev7d,min_temperature_prev7d,...,skew_equipment_alarms_prev14d,skew_fire/smoke_alarms_prev14d,skew_ge_alarms_prev14d,skew_power_alarms_prev14d,skew_temperature_alarms_prev14d,kurt_equipment_alarms_prev14d,kurt_fire/smoke_alarms_prev14d,kurt_ge_alarms_prev14d,kurt_power_alarms_prev14d,kurt_temperature_alarms_prev14d
0,858,2019-04-10,10.0,0,0.0,0.0,12.00,8,16.0,9.0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.210000,-1.212308,-1.21,-1.210000
1,858,2019-04-11,10.0,0,0.0,0.0,13.14,8,17.0,10.0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.210000,-1.212308,-1.21,-1.210000
2,858,2019-04-12,10.0,0,0.0,0.0,13.29,8,17.0,11.0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.210000,-1.212308,-1.21,-1.210000
3,858,2019-04-13,10.0,0,0.0,0.0,13.00,8,17.0,9.0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.210000,-1.212308,-1.21,-1.210000
4,858,2019-04-14,10.0,0,0.0,0.0,12.43,8,17.0,8.0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.210000,-1.212308,-1.21,-1.210000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160195,25,2020-01-30,1.0,0,0.0,0.0,3.86,9,7.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.21,-1.212308
160196,25,2020-01-31,1.0,0,0.0,0.0,4.14,9,7.0,1.0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.21,-1.212308
160197,25,2020-02-01,1.0,0,0.0,0.0,4.29,9,7.0,1.0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.21,-1.212308
160198,25,2020-02-02,1.0,0,0.0,0.0,4.86,9,9.0,1.0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.21,-1.212308
